In [ ]:
#Hier werden die Szenearien in fünfer Schritten durchgegangen um besser zu sehen ab wann das system kippt

import simpy
import numpy as np
import pandas as pd

# =========================================================
# 1. KONFIGURATION
# =========================================================
TOTAL_BEDS = 447
BASE_SEED = 42
R = 20                      # Monte-Carlo-Replikationen
SIM_DURATION = 150          # Simulationsdauer (Tage)
WARMUP_OCCUPANCY = 0.75     # 75 % Vorbelegung
MEAN_LOS = 14               # mittlere Aufenthaltsdauer (Tage)

# =========================================================
# 2. PATIENTENPROZESS (DES)
# =========================================================
def patient_process(env, hospital, mean_los, rng, stats, is_preloaded=False):
    arrival_time = env.now

    with hospital.request() as req:
        yield req
        start_service = env.now

        # Statistik nur für reguläre Patienten
        if not is_preloaded:
            stats["wait_times"].append(start_service - arrival_time)

        # Aufenthaltsdauer
        yield env.timeout(rng.exponential(mean_los))

# =========================================================
# 3. KRANKENHAUS-SETUP (INKL. WARM-UP)
# =========================================================
def setup_hospital(env, hospital, rng, stats, patients_per_week, mean_los):
    mean_interarrival = 7.0 / patients_per_week

    # Warm-up: Anfangsbelegung
    preload = int(TOTAL_BEDS * WARMUP_OCCUPANCY)
    for _ in range(preload):
        env.process(patient_process(
            env, hospital, mean_los, rng, stats, is_preloaded=True
        ))

    # Laufender Ankunftsprozess
    while True:
        yield env.timeout(rng.exponential(mean_interarrival))
        env.process(patient_process(
            env, hospital, mean_los, rng, stats
        ))

# =========================================================
# 4. ENGPass-MONITOR (ZIELEREIGNIS Yi)
# =========================================================
def monitor_system(env, hospital, stats):
    """
    Engpass liegt vor, wenn:
    - Warteschlange > 0 ODER
    - Belegung == Kapazität
    """
    last_time = env.now

    while True:
        yield env.timeout(0.1)  # feine Zeitauflösung
        now = env.now
        dt = now - last_time
        last_time = now

        queue_len = len(hospital.queue)
        occupancy = hospital.count

        if queue_len > 0 or occupancy >= TOTAL_BEDS:
            stats["engpass_time"] += dt
            stats["engpass_flag"] = 1  # Yi = 1

# =========================================================
# 5. EIN SIMULATIONSLauf
# =========================================================
def run_single_simulation(patients_per_week, mean_los, seed):
    rng = np.random.default_rng(seed)
    env = simpy.Environment()
    hospital = simpy.Resource(env, capacity=TOTAL_BEDS)

    stats = {
        "wait_times": [],
        "engpass_time": 0.0,
        "engpass_flag": 0
    }

    env.process(setup_hospital(
        env, hospital, rng, stats, patients_per_week, mean_los
    ))
    env.process(monitor_system(env, hospital, stats))

    env.run(until=SIM_DURATION)
    return stats

# =========================================================
# 6. MONTE-CARLO-SCHLEIFE
# =========================================================
def monte_carlo(patients_per_week, mean_los):
    wait_means = []
    engpass_flags = []
    engpass_times = []

    for i in range(R):
        stats = run_single_simulation(
            patients_per_week, mean_los, BASE_SEED + i
        )

        if stats["wait_times"]:
            wait_means.append(np.mean(stats["wait_times"]))

        engpass_flags.append(stats["engpass_flag"])
        engpass_times.append(stats["engpass_time"] / SIM_DURATION)

    mean_wait = np.mean(wait_means)
    se_wait = np.std(wait_means, ddof=1) / np.sqrt(R)
    ci_wait = (mean_wait - 1.96 * se_wait, mean_wait + 1.96 * se_wait)

    p_engpass = np.mean(engpass_flags)
    mean_engpass_time = np.mean(engpass_times)

    return mean_wait, se_wait, ci_wait, p_engpass, mean_engpass_time

# =========================================================
# 7. LASTSTEIGERUNGS-SZENARIO (5er-Schritte)
# =========================================================
patients_range = range(35, 236, 5)  # 35 → 235 in 5er-Schritten
results = []

for p_week in patients_range:
    m, se, ci, p_e, t_e = monte_carlo(p_week, MEAN_LOS)
    rho = ((p_week / 7) * MEAN_LOS) / TOTAL_BEDS

    results.append({
        "Pat./Woche": p_week,
        "Auslastung ρ": rho,
        "Ø Wartezeit (d)": m,
        "SE": se,
        "95%-KI": f"[{ci[0]:.2f}, {ci[1]:.2f}]",
        "P(Engpass)": p_e,
        "Zeit im Engpass (%)": t_e * 100
    })

# =========================================================
# 8. OUTPUT & SCHWELLENIDENTIFIKATION
# =========================================================
df = pd.DataFrame(results)

df["Belastungsstatus"] = df["P(Engpass)"].apply(
    lambda x: "OK" if x < 0.1 else
              "kritisch" if x < 0.5 else
              "Überlastet"
)

print("\nLASTSTEIGERUNGS-ANALYSE (DES + MONTE CARLO)")
print("=" * 120)
print(df.to_string(index=False, formatters={
    "Auslastung ρ": lambda x: f"{x*100:.1f}%",
    "Ø Wartezeit (d)": lambda x: f"{x:.3f}",
    "SE": lambda x: f"{x:.3f}",
    "P(Engpass)": lambda x: f"{x:.2f}",
    "Zeit im Engpass (%)": lambda x: f"{x:.1f}"
}))
print("=" * 120)

# Automatische Belastungsschwelle
critical = df[df["P(Engpass)"] >= 0.5]

if not critical.empty:
    first = critical.iloc[0]
    print(
        f"\n⚠️ Belastungsschwelle ab ca. "
        f"{first['Pat./Woche']} Patienten/Woche "
        f"(P(Engpass) ≥ 50%)"
    )
else:
    print("\n✅ Keine kritische Belastung im untersuchten Bereich.")

print(
    f"\nParameter: Betten={TOTAL_BEDS}, LOS={MEAN_LOS} Tage, "
    f"R={R}, Dauer={SIM_DURATION} Tage"
)
